<a href="https://colab.research.google.com/github/tedheo10/STATS-504/blob/main/transstats_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
library(tidyverse)
library(ggplot2)
install.packages("nycflights13")
library(nycflights13)
options(repr.plot.width = 10, repr.plot.height = 6)
# httr::set_config(config(ssl_verifypeer = 0L))
# -k means don't verify cert, -L means follow redirects (important for some servers)
options(download.file.method="curl", download.file.extra="-k -L")
theme_set(theme_minimal())

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# STATS 504
## Week 1: TransStats data

## TransStats
- [U.S. Bureau of Transportation Statistics](http://www.transtats.bts.gov)
- Aviation, maritime, highway, transit, and rail.
- We will focus on aviation today

## Three questions for lecture
For today's lecture I want to answer the following questions:

- Which airlines are likely to cancel a flight during spring break?
- Where do people fly to from Detroit in the winter?

## Getting the data
- The web site has interactive query builders for the different data bases.
- Example: [airline on-time data](https://www.transtats.bts.gov/Tables.asp?QO_VQ=EFD&QO_anzr=Nv4yv0r%FDb0-gvzr%FDcr4s14zn0pr%FDQn6n&QO_fu146_anzr=b0-gvzr)
- The interface lets you download one month of data at a time -- how to automate?

In [ ]:
base <- 'https://www.transtats.bts.gov/PREZIP/'

template <- paste0(base, 'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip')


template %>% str_glue(year=2020, month=3)

https://www.transtats.bts.gov/PREZIP/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2020_3.zip

In [ ]:
template %>% str_glue(year=2022, month=1) %>% download.file("2022_01.zip")

In [ ]:
df = read.csv("2022_01.zip")
df %>% head

## Assembling the dataset
Let's download all the data from 2022 (the most recent complete year).


Each `.zip` file contains a README describing the column layout. From this, I build the following column specification to extract only the data I need:

In [ ]:
columns <- cols_only(
  FlightDate = col_datetime(),
  Tail_Number = col_character(),
  Reporting_Airline = col_character(),
  Origin = col_character(),
  Dest = col_character(),
  DepTime = col_double(),
  DepDelay = col_double(),
  ArrTime = col_double(),
  ArrDelay = col_double(),
  Cancelled = col_double(),
  CancellationCode = col_character(),
  AirTime = col_double(),
  Distance = col_double(),
  CarrierDelay = col_double(),
  WeatherDelay = col_double(),
  NASDelay = col_double(),
  SecurityDelay = col_double(),
  LateAircraftDelay = col_double()
)

## Data download function
- Now I want to download the data for 2022-1, 2022-2, ..., 2022-12 in succession and paste them together.
- Since we're going to be applying the same code twelve different times, it makes sense to package this code up into a function and re-use it.
- Guiding principle: don't repeat yourself! ([DRY](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself))

In [ ]:
dl_data <- function(month, year) {
    fn <- paste0(year, "_", month, ".zip")
    if (! file.exists(fn)) {
        str_glue(template, year=year, month=month) %>% download.file(fn)
    }
    return(read_csv(fn, col_types = columns))
}

In [ ]:
load("flights_2022.RData")

In [ ]:
flights_2022 %>% print

## Who is most likely to cancel flights

- I am worried about my flight getting cancelled. Which air carrier is most likely to cancel?
- How should we we measure? Ideas:
    - Total Cancellations / Total Scheduled Flights.
    - Some cancellations are due to circumstances beyond the carrier's control (e.g. weather).
    - Route specific cancellations: are some routes more likely to get cancelled?
    - Benchmark: compare an airline's cancellation rate against the industry average.

## Basic analysis
Let us start by analyzing the overall cancellation rate.

In [ ]:
nycflights13::airlines %>% print

In [ ]:
flights_2022 %>% group_by(Reporting_Airline) %>%
    summarise(cr = mean(Cancelled), n=n()) %>%
    left_join

This has revealed a problem in our data. The join did not perform as we expected.

Carrier's names are missing

Incidentally, this is an argument for using `left_join` instead of `inner_join`. Inner join would have silently eliminated all the non-matching rows. We would have no idea that we were missing out on hundreds of thousands of flights.

## Scraping
We need a quick way to get an up-to-date database of airline codes.
- Wikipedia has a list [here](https://en.wikipedia.org/wiki/List_of_airline_codes).
- How can we get this into R?

In [ ]:
#install.packages("rvest")
library(rvest)
read_html('https://en.wikipedia.org/wiki/List_of_airline_codes') %>% html_table %>% .[[1]] ->
    airline_codes

In [ ]:
airline_codes %>% print

In [ ]:
flights_2022 %>% left_join(airline_codes, join_by(Reporting_Airline == IATA)) %>%
  group_by(Airline) %>% summarize(cr = mean(Canceled)) %>%


What about the warning we received?

## Cancellation rate by week
- Let's consider cancellations by week of the year.
- Are cancellations more likely during some periods of the year than others?

This plot is sort of hard to read, so let's think of same ways to improve it:
- Plot carrier name instead of code.
- Consider deviation from average weekly delay.

(What is the [spike](https://en.wikipedia.org/wiki/2022_Southwest_Airlines_scheduling_crisis) at the very end of 2022?)

The plot is still rather jumbled. We can try zooming in on specific carriers:

## Reasons for cancellation

- For flights that are cancelled, there is a code:

- [What do the codes mean](https://www.transtats.bts.gov/FieldInfo.asp?Svryq_Qr5p=f2rpvsvr5%FDgur%FDern510%FDS14%FDPn0pryyn6v10&Svryq_gB2r=Pun4&Y11x72_gnoyr=Y_PNaPRYYNgVba&gnoyr_VQ=FGK&flf_gnoyr_anzr=g_bagVZR_ZNeXRgVaT&fB5_Svryq_anzr=PNaPRYYNgVba_PbQR)?

- Now we want to visualize the distribution of two categorical variables: `airline` and `cancellation code`.
- How should we do it?
- (Recall last lecture)

To make the table more readable, we should normalize the table across:
    - Rows?
    - Columns?

## Where do people in Michigan fly during the winter?

To study this, we will use a random 10% sample of all airline tickets issued during Q12023:

https://www.transtats.bts.gov/DatabaseInfo.asp?QO_VQ=EFI&Yv0x=D

In [ ]:
load("tkt_2023.RData")

In [ ]:
states_abbrs <- data.frame(
  Abbreviation = c("AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
                   "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
                   "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
                   "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
                   "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"),
  StateName = c("Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
                "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho",
                "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
                "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
                "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
                "New Hampshire", "New Jersey", "New Mexico", "New York",
                "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon",
                "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
                "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
                "West Virginia", "Wisconsin", "Wyoming")
)

#print(states_abbrs)
